
Clustering & Fitting Assignment

Reads:  owid-co2-data.csv

Writes: outputs/ (plots, CSVs, metrics, short assessment)



#### Load Libraries

In [2]:

import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [3]:

# ---------------- Config ----------------
DATA_PATH = "owid-co2-data.csv"   # uploaded dataset path
OUT_DIR = "outputs"
os.makedirs(OUT_DIR, exist_ok=True)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
sns.set(style="whitegrid")


#### Load and inspect dataset

In [4]:

# ---------------- 1. Load and inspect dataset ----------------
print("Loading dataset from:", DATA_PATH)
df = pd.read_csv(DATA_PATH)
print("Raw shape:", df.shape)
print("Columns:", df.columns.tolist())


Loading dataset from: owid-co2-data.csv
Raw shape: (25204, 58)
Columns: ['iso_code', 'country', 'year', 'co2', 'consumption_co2', 'co2_growth_prct', 'co2_growth_abs', 'trade_co2', 'co2_per_capita', 'consumption_co2_per_capita', 'share_global_co2', 'cumulative_co2', 'share_global_cumulative_co2', 'co2_per_gdp', 'consumption_co2_per_gdp', 'co2_per_unit_energy', 'coal_co2', 'cement_co2', 'flaring_co2', 'gas_co2', 'oil_co2', 'other_industry_co2', 'cement_co2_per_capita', 'coal_co2_per_capita', 'flaring_co2_per_capita', 'gas_co2_per_capita', 'oil_co2_per_capita', 'other_co2_per_capita', 'trade_co2_share', 'share_global_cement_co2', 'share_global_coal_co2', 'share_global_flaring_co2', 'share_global_gas_co2', 'share_global_oil_co2', 'share_global_other_co2', 'cumulative_cement_co2', 'cumulative_coal_co2', 'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_oil_co2', 'cumulative_other_co2', 'share_global_cumulative_cement_co2', 'share_global_cumulative_coal_co2', 'share_global_cumulati

#### Filter data (1990-2019) and clean

In [6]:
# ---------------- 2. Filter data (1990-2019) and clean ----------------
YEARS = list(range(1990, 2020))
if "year" in df.columns:
    df = df[df["year"].isin(YEARS)].copy()
else:
    raise RuntimeError("'year' column not found in dataset.")

# Remove OWID aggregates if present
if "iso_code" in df.columns:
    df = df[~df["iso_code"].isna()]
    df = df[~df["iso_code"].astype(str).str.startswith("OWID_")]

# Select commonly-used columns that we will reference; only keep those present
wanted = [
    "iso_code", "country", "year",
    "co2", "co2_per_capita", "consumption_co2", "consumption_co2_per_capita",
    "population", "gdp", "total_ghg", "ghg_per_capita",
    "methane", "nitrous_oxide", "primary_energy_consumption",
    "energy_per_capita", "energy_per_gdp"
]
available = [c for c in wanted if c in df.columns]
df = df[available].copy()
# require co2_per_capita for analyses
if "co2_per_capita" not in df.columns:
    raise RuntimeError("co2_per_capita column not found. Cannot continue.")

df = df.dropna(subset=["co2_per_capita"]).reset_index(drop=True)
print("Filtered shape (1990-2019, with co2_per_capita):", df.shape)


Filtered shape (1990-2019, with co2_per_capita): (6401, 16)


#### Compute country-level moments (four main moments) 

In [7]:
# ---------------- 3. Compute country-level moments (four main moments) ----------------
# We'll group by iso_code (if available) and country
group_cols = ["iso_code", "country"] if "iso_code" in df.columns else ["country"]
grouped = df.groupby(group_cols, dropna=False)

def compute_moments(series):
    arr = series.dropna().values
    n = len(arr)
    if n == 0:
        return pd.Series({
            "count": 0, "mean_co2pc": np.nan, "var_co2pc": np.nan,
            "std_co2pc": np.nan, "skew_co2pc": np.nan, "kurt_co2pc": np.nan
        })
    mean = float(arr.mean())
    var = float(arr.var(ddof=0))        # population variance
    std = float(arr.std(ddof=1)) if n > 1 else 0.0
    skew = float(stats.skew(arr, bias=False)) if n > 2 else np.nan
    kurt = float(stats.kurtosis(arr, fisher=True, bias=False)) if n > 3 else np.nan
    return pd.Series({
        "count": n, "mean_co2pc": mean, "var_co2pc": var,
        "std_co2pc": std, "skew_co2pc": skew, "kurt_co2pc": kurt
    })

country_stats = grouped["co2_per_capita"].apply(compute_moments).reset_index()
# if gdp present, compute mean gdp per country for clustering
if "gdp" in df.columns:
    gdp_mean = grouped["gdp"].mean().reset_index().rename(columns={"gdp": "mean_gdp"})
    country_stats = country_stats.merge(gdp_mean, on=group_cols, how="left")

country_stats.to_csv(os.path.join(OUT_DIR, "country_moments.csv"), index=False)
print("Saved country-level moments ->", os.path.join(OUT_DIR, "country_moments.csv"))


Saved country-level moments -> outputs\country_moments.csv


#### Plots (relational, categorical, statistical) 

In [8]:
# ---------------- 4. Plots (relational, categorical, statistical) ----------------
# Create a snapshot for 2019 if possible, else use the latest available per country
df2019 = df[df["year"] == 2019].copy()
if df2019.empty:
    # fallback: last available record per country
    last_year = df.groupby(group_cols)["year"].transform("max")
    df2019 = df[df["year"] == last_year].copy()
print("2019 snapshot shape (or last available):", df2019.shape)

# 4a Relational plot: GDP vs CO2 per capita (scatter + regression)
plt.figure(figsize=(10, 6))
if "gdp" in df2019.columns and df2019["gdp"].notna().sum() >= 5:
    # log transform to reduce skew
    sns.regplot(x=np.log1p(df2019["gdp"]), y=df2019["co2_per_capita"],
                scatter_kws={"alpha": 0.6}, line_kws={"color":"red"})
    plt.xlabel("log(1 + GDP) (2019)")
else:
    # no reliable GDP, use index on x-axis
    sns.regplot(x=np.arange(len(df2019)), y=df2019["co2_per_capita"], scatter_kws={"alpha": 0.6})
    plt.xlabel("Country index")
plt.ylabel("CO2 per capita")
plt.title("Relational: GDP vs CO2 per capita (2019 snapshot)")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "relational_gdp_vs_co2_2019.png"))
plt.close()
print("Saved relational plot -> relational_gdp_vs_co2_2019.png")

# 4b Categorical plot: Top-10 countries by total CO2 (2019)
if "co2" in df2019.columns:
    top10 = df2019[["country", "co2"]].dropna().sort_values("co2", ascending=False).head(10)
    plt.figure(figsize=(10, 6))
    sns.barplot(data=top10, x="co2", y="country")
    plt.xlabel("Total CO2 (2019)")
    plt.title("Top 10 countries by total CO2 (2019)")
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, "categorical_top10_co2_2019.png"))
    plt.close()
    print("Saved categorical plot -> categorical_top10_co2_2019.png")
else:
    # fallback categorical: top by co2_per_capita
    top10 = df2019[["country", "co2_per_capita"]].dropna().sort_values("co2_per_capita", ascending=False).head(10)
    plt.figure(figsize=(10, 6))
    sns.barplot(data=top10, x="co2_per_capita", y="country")
    plt.xlabel("CO2 per capita (2019)")
    plt.title("Top 10 countries by CO2 per capita (2019)")
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, "categorical_top10_co2pc_2019.png"))
    plt.close()
    print("Saved categorical plot -> categorical_top10_co2pc_2019.png")

# 4c Statistical plots:
# Correlation heatmap for numeric features present
numeric_candidates = ["co2_per_capita", "gdp", "population", "co2", "total_ghg", "ghg_per_capita"]
num_cols = [c for c in numeric_candidates if c in df2019.columns]
if len(num_cols) >= 2:
    corr = df2019[num_cols].corr()
    plt.figure(figsize=(6, 5))
    sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm")
    plt.title("Correlation heatmap (2019 snapshot)")
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, "heatmap_2019.png"))
    plt.close()
    print("Saved heatmap -> heatmap_2019.png")
else:
    print("Not enough numeric columns for heatmap. Skipping.")

# Boxplot: use GDP-deciles as a categorical grouping if GDP is usable
if "gdp" in df2019.columns and df2019["gdp"].notna().sum() >= 10 and df2019["gdp"].nunique() >= 10:
    # build deciles from rank to avoid ties causing qcut failure; duplicates='drop' is extra safety
    try:
        # work on a copy
        df2019_cp = df2019.copy()
        df2019_cp["gdp_decile"] = pd.qcut(df2019_cp["gdp"].rank(method="first"),
                                          q=10, labels=False, duplicates="drop") + 1
        plt.figure(figsize=(12, 6))
        sns.boxplot(data=df2019_cp.dropna(subset=["gdp_decile"]), x="gdp_decile", y="co2_per_capita")
        plt.xlabel("GDP decile (1 = lowest)")
        plt.title("CO2 per capita by GDP decile (2019)")
        plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, "boxplot_co2_by_gdp_decile_2019.png"))
        plt.close()
        print("Saved boxplot -> boxplot_co2_by_gdp_decile_2019.png")
    except Exception as e:
        print("Could not create GDP-decile boxplot (qcut issue):", str(e))
else:
    print("Skipping GDP-decile boxplot: not enough distinct GDP values.")

# Pairplot of the four moments (safe)
moment_cols = ["mean_co2pc", "std_co2pc", "skew_co2pc", "kurt_co2pc"]
missing_moments = [c for c in moment_cols if c not in country_stats.columns]
if missing_moments:
    # Try to compute simpler moments (fallback)
    print("Missing moment columns:", missing_moments, " — attempting alternative aggregation.")
    # compute using agg shortcuts
    fallback = df.groupby(group_cols)["co2_per_capita"].agg(
        mean_co2pc="mean", std_co2pc="std"
    ).reset_index()
    # compute skew & kurt per-country using apply where possible
    if "skew_co2pc" not in fallback.columns:
        skew = df.groupby(group_cols)["co2_per_capita"].apply(lambda x: float(stats.skew(x.dropna(), bias=False)) if len(x.dropna())>2 else np.nan).reset_index(name="skew_co2pc")
        kurt = df.groupby(group_cols)["co2_per_capita"].apply(lambda x: float(stats.kurtosis(x.dropna(), fisher=True, bias=False)) if len(x.dropna())>3 else np.nan).reset_index(name="kurt_co2pc")
        fallback = fallback.merge(skew, on=group_cols, how="left").merge(kurt, on=group_cols, how="left")
    pair_df = fallback[["mean_co2pc", "std_co2pc", "skew_co2pc", "kurt_co2pc"]].dropna()
else:
    pair_df = country_stats[moment_cols].dropna()

if len(pair_df) > 1:
    sample = pair_df.sample(min(200, len(pair_df)), random_state=RANDOM_STATE)
    try:
        sns.pairplot(sample)
        plt.suptitle("Pairplot: Moments of CO2 per capita (sample)", y=1.02)
        plt.savefig(os.path.join(OUT_DIR, "pairplot_moments.png"))
        plt.close()
        print("Saved pairplot -> pairplot_moments.png")
    except Exception as e:
        print("Pairplot failed:", e)
else:
    print("Not enough country-moment rows to produce a pairplot. Skipping.")

# Save a descriptive summary for co2_per_capita
desc = df["co2_per_capita"].describe()
with open(os.path.join(OUT_DIR, "descriptive_co2_per_capita.txt"), "w") as fh:
    fh.write(str(desc))
print("Saved descriptive stats -> descriptive_co2_per_capita.txt")


2019 snapshot shape (or last available): (214, 16)
Saved relational plot -> relational_gdp_vs_co2_2019.png
Saved categorical plot -> categorical_top10_co2_2019.png
Saved heatmap -> heatmap_2019.png
Skipping GDP-decile boxplot: not enough distinct GDP values.
Missing moment columns: ['mean_co2pc', 'std_co2pc', 'skew_co2pc', 'kurt_co2pc']  — attempting alternative aggregation.
Saved pairplot -> pairplot_moments.png
Saved descriptive stats -> descriptive_co2_per_capita.txt


####  Clustering (KMeans)

In [9]:
# ---------------- 5. Clustering (KMeans) ----------------
# Prepare features: mean_co2pc, std_co2pc, optionally mean_gdp
cluster_cols = [c for c in ["mean_co2pc", "std_co2pc", "mean_gdp"] if c in country_stats.columns]

if not cluster_cols:
    print("No clustering features available (mean_co2pc, std_co2pc, mean_gdp missing). Skipping clustering.")
else:
    cluster_df = country_stats.copy()
    # Report missing columns if any
    missing_cols = [c for c in ["mean_co2pc", "std_co2pc", "mean_gdp"] if c not in cluster_df.columns]
    if missing_cols:
        print(f"Missing clustering columns: {missing_cols} — clustering will proceed with available features: {cluster_cols}")

    # Drop rows with missing clustering values
    cluster_df_num = cluster_df.dropna(subset=cluster_cols)
    if cluster_df_num.empty:
        print("No countries with complete clustering features — skipping clustering.")
    else:
        cluster_df_num = cluster_df_num.set_index(group_cols)
        X = cluster_df_num[cluster_cols].values

        # Standardize features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # Elbow plot
        inertia = []
        Ks = list(range(1, 11))
        for k in Ks:
            km = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
            km.fit(X_scaled)
            inertia.append(km.inertia_)
        plt.figure(figsize=(8, 5))
        plt.plot(Ks, inertia, "-o")
        plt.xlabel("k")
        plt.ylabel("Inertia")
        plt.title("Elbow plot for KMeans (country-level features)")
        plt.xticks(Ks)
        plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, "kmeans_elbow.png"))
        plt.close()
        print("Saved elbow plot -> kmeans_elbow.png")

        # Silhouette scores for k=2..6
        sil_scores = {}
        for k in range(2, 7):
            km = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
            labels = km.fit_predict(X_scaled)
            sil_scores[k] = silhouette_score(X_scaled, labels)
        best_k = max(sil_scores, key=sil_scores.get)

        # Final KMeans with best k
        km = KMeans(n_clusters=best_k, random_state=RANDOM_STATE, n_init=20)
        labels = km.fit_predict(X_scaled)
        cluster_df_num["kmeans_cluster"] = labels

        cluster_assignments_path = os.path.join(OUT_DIR, "country_clusters_kmeans.csv")
        cluster_df_num.reset_index().to_csv(cluster_assignments_path, index=False)
        print(f"Saved cluster assignments -> {cluster_assignments_path} (best_k={best_k})")

        # PCA projection visualization (only if enough samples/features)
        if cluster_df_num.shape[0] > 1 and len(cluster_cols) > 1:
            pca = PCA(n_components=2, random_state=RANDOM_STATE)
            proj = pca.fit_transform(X_scaled)
            plt.figure(figsize=(8, 6))
            sns.scatterplot(
                x=proj[:, 0], y=proj[:, 1],
                hue=labels,
                palette=sns.color_palette(n_colors=best_k),
                s=60
            )
            plt.title(f"KMeans clusters (k={best_k}) projected to 2D (PCA)")
            plt.xlabel("PC1"); plt.ylabel("PC2")
            plt.tight_layout()
            plt.savefig(os.path.join(OUT_DIR, f"kmeans_k{best_k}_pca.png"))
            plt.close()
            print(f"Saved PCA cluster plot -> kmeans_k{best_k}_pca.png")
        else:
            print("Skipping PCA: not enough rows or features for 2D projection.")


Missing clustering columns: ['mean_co2pc', 'std_co2pc'] — clustering will proceed with available features: ['mean_gdp']
Saved elbow plot -> kmeans_elbow.png
Saved cluster assignments -> outputs\country_clusters_kmeans.csv (best_k=2)
Skipping PCA: not enough rows or features for 2D projection.


#### Fitting (Regression)

In [10]:
# ---------------- 6. Fitting (Regression) ----------------
# Predict co2_per_capita from log(GDP) and log(population) using 2010-2019 rows
fit_df = df[(df["year"] >= 2010) & (df["year"] <= 2019)].dropna(subset=["co2_per_capita"]).copy()
reg_metrics_path = os.path.join(OUT_DIR, "regression_metrics.txt")

if ("gdp" in fit_df.columns) and ("population" in fit_df.columns) and (fit_df[["gdp", "population"]].dropna().shape[0] > 10):
    fit_df = fit_df.dropna(subset=["gdp", "population"])
    fit_df["log_gdp"] = np.log1p(fit_df["gdp"])
    fit_df["log_pop"] = np.log1p(fit_df["population"])
    Xr = fit_df[["log_gdp", "log_pop"]].values
    yr = fit_df["co2_per_capita"].values

    X_train, X_test, y_train, y_test = train_test_split(Xr, yr, test_size=0.2, random_state=RANDOM_STATE)

    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred_lr = lr.predict(X_test)

    ridge = Ridge(alpha=1.0)
    ridge.fit(X_train, y_train)
    y_pred_ridge = ridge.predict(X_test)

    def metrics(y_true, y_pred):
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        return {"mse": mse, "rmse": rmse, "mae": mae, "r2": r2}

    lr_metrics = metrics(y_test, y_pred_lr)
    ridge_metrics = metrics(y_test, y_pred_ridge)

    with open(reg_metrics_path, "w") as fh:
        fh.write("Linear Regression metrics:\n")
        fh.write(str(lr_metrics) + "\n\n")
        fh.write("Ridge Regression metrics:\n")
        fh.write(str(ridge_metrics) + "\n")

    # plot actual vs predicted (Ridge)
    plt.figure(figsize=(7, 6))
    plt.scatter(y_test, y_pred_ridge, alpha=0.6)
    mn = min(np.nanmin(y_test), np.nanmin(y_pred_ridge))
    mx = max(np.nanmax(y_test), np.nanmax(y_pred_ridge))
    plt.plot([mn, mx], [mn, mx], "r--")
    plt.xlabel("Actual CO2 per capita")
    plt.ylabel("Predicted CO2 per capita (Ridge)")
    plt.title("Actual vs Predicted CO2 per capita (Ridge)")
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, "regression_actual_vs_predicted_ridge.png"))
    plt.close()

    print("Saved regression metrics ->", reg_metrics_path)
    print("Saved regression plot -> regression_actual_vs_predicted_ridge.png")
else:
    with open(reg_metrics_path, "w") as fh:
        fh.write("Not enough data to run regression (missing gdp or population or insufficient rows)\n")
    print("Regression skipped: insufficient data (gdp/population). Metrics file written stating skip.")


Saved regression metrics -> outputs\regression_metrics.txt
Saved regression plot -> regression_actual_vs_predicted_ridge.png


#### Short assessment saved

In [11]:
# ---------------- 7. Short assessment saved ----------------
with open(os.path.join(OUT_DIR, "assessment.txt"), "w") as fh:
    fh.write("Clustering & Fitting Assignment - Short assessment\n\n")
    fh.write("- Dataset: uploaded OWID CO2 CSV filtered to 1990-2019.\n")
    fh.write("- Computed per-country four main moments for co2_per_capita and saved to country_moments.csv.\n")
    fh.write("- Created relational (GDP vs CO2pc), categorical (top-10 CO2), and statistical plots (heatmap, boxplot by GDP-decile where possible, pairplot sample).\n")
    fh.write("- Performed KMeans clustering on country-level statistics; selected k by silhouette score and saved assignments.\n")
    fh.write("- Fitted LinearRegression and Ridge to predict CO2 per capita from log(GDP) and log(population) for 2010-2019; metrics saved.\n")
    fh.write("\nLimitations:\n- No continent column in uploaded data; used GDP-deciles as categorical grouping for boxplot.\n- Missing values reduce sample size for some countries and analyses.\n")
print("Saved short assessment -> assessment.txt")


Saved short assessment -> assessment.txt
